In [ ]:
from init_notebook import *
from src.algo.sdf.two_d.util import *
from src.algo.sdf.two_d import *

In [ ]:
class Tiles:
    L = 1
    T = 1 << 1
    R = 1 << 2
    B = 1 << 3

    TL = 1 << 4
    TR = 1 << 5
    BR = 1 << 6
    BL = 1 << 7

    ALL_EDGES = L|R|T|B
    ALL_CORNERS = TL|TR|BL|BR

    # 4x4
    EDGE_ORDER = [
        [0, R, L|R, L],
        [B, B|R, B|L|R, B|L],
        [T|B, T|B|R, T|B|L|R, T|B|L],
        [T, T|R, T|L|R, T|L],
    ]

    # 4x4
    CORNER_ORDER = [
        [0, TR, TL|BR, BL],
        [BR, BL|BR, BL|BR|TR, BL|TL],
        [BL|TR, TL|TR|BR, TL|TR|BL|BR, TL|BL|BR],
        [TL, TR|BR, TL|TR|BL, TL|TR], 
    ]

    # 7x7
    EDGE_AND_CORNER_ORDER = [
        [TL|L|BL|T|B|TR|R|BR, TL|T|TR|L|BL|B|BR, TL|T|TR|BL, TL|T|TR, TL|T|TR|BR, TL|T|TR|R|BR|BL, TL|L|BL|T|B|TR|R|BR],
        [BL|L|TL|T|TR|R|BR, BL|L|TL|T|TR|BR, TL|BL|BR, BL, TR|R|BR, TL|L|BL|TR|BR, TL|T|TR|R|BR|B|BL],
        [TL|L|BL|TR, TL|TR|BR, BL|TL|TR|BR, TL|BL|B|BR, BL|TR, TL|TR, TL|T|TR|R|BR],  
        [TL|L|BL, TR, TL|TR|R|BR, BL|L|TL|T|TR, TL, BR, BL|TR|R|BR],
        [TL|L|BL|BR, BL|B|BR, BL|BR|TR, TL|BL, 0, TR|BR, TL|BL|TR|R|BR],
        [TL|L|BL|B|BR|TR, TL|T|TR|BL|BR, TL|TR|BL, TL|BR, BL|BR, BL|B|BR|R|TR, TL|L|BL|TR|R|BR],
        [TL|L|BL|T|B|TR|R|BR, TL|L|BL|B|BR|R|TR, TL|L|BL|B|BR, BL|B|BR|TR, TL|TR|BL|B|BR, TL|T|TR|BL|B|BR, TL|BL|B|BR|R|TR], 
    ]

    EO2 = [
        [0, R, L|R|B|BR, L|R|BL|B|BR, L|R|BL|B, L|B, 0], 
        [B, R|B, T|TR|L|R|B, TL|T|TR|L|R|B|BR, TL|T|L|BL|B, T|R|B, L], 
        [T|R|B|BR, T|L|R|BL|B, T|L|R|B, T|TR|L|R, TL|T|L|R|B|BR, T|L|R|BL|B|BR, L|BL|B], 
        [T|TR|R|B|BR, TL|T|L|R|BL|B|BR, T|L|BL|B, R|B|BR, T|TR|L|R|BL|B|BR, TL|T|TR|L|R|BL|B, TL|T|L|B], 
        [T|TR|R|B, TL|T|TR|L|R, TL|T|L|R|B, T|TR|L|R|B|BR, TL|T|TR|L|R|BL|B|BR, TL|T|L|R|BL|B, T|L|B], 
        [T|R, L|R|B, T|L|R|B|BR, T|TR|L|R|BL|B, TL|T|TR|L|R|B, TL|T|L, T|B], 
        [0, T, T|TR|R, TL|T|L|R, T|L|R, L|R, T|L],
    ]

    @classmethod
    def edges(cls, index: int, radius: float = 0.1):
        if index & cls.ALL_EDGES == cls.ALL_EDGES:
            yield Box((2, 2))
        if index & cls.L:
            yield Box((1, radius)).translate((.5, 0))
        if index & cls.R:
            yield Box((1, radius)).translate((.5, 1))
        if index & cls.T:
            yield Box((radius, 1)).translate((0, .5))
        if index & cls.B:
            yield Box((radius, 1)).translate((1, .5))

    @classmethod
    def corners(cls, index: int, radius: float = 0.1):
        box = Circle(radius)
        if index & cls.TL:
            yield box
        if index & cls.TR:
            yield box.translate((0, 1))
        if index & cls.BL:
            yield box.translate((1, 0))
        if index & cls.BR:
            yield box.translate((1, 1))

    @classmethod
    def edge_circles(cls, index: int, radius: float = 0.1):
        if index & (cls.T|cls.L) == (cls.T|cls.L):
            yield Circle(1-radius).invert().translate((1, 1))
        if index & (cls.T|cls.R) == (cls.T|cls.R):
            yield Circle(1-radius).invert().translate((1, 0))
        if index & (cls.B|cls.L) == (cls.B|cls.L):
            yield Circle(1-radius).invert().translate((0, 1))
        if index & (cls.B|cls.R) == (cls.B|cls.R):
            yield Circle(1-radius).invert().translate((0, 0))

    @classmethod
    def edge_triangles(cls, index: int, radius: float=.25, radius2: float = .5):
        o = Box(radius*np.sqrt(2)).rotate(45)
        io = Box((radius2, 2))
        if index & cls.ALL_EDGES == cls.ALL_EDGES:
            yield Box(.5).translate(.5)
        if index & cls.L:
            yield o.intersect(io.rotate(90)).translate((.5, 0))
        if index & cls.R:
            yield o.intersect(io.rotate(90)).translate((.5, 1))
        if index & cls.T:
            yield o.intersect(io).translate((0, .5))
        if index & cls.B:
            yield o.intersect(io).translate((1, .5))

    @classmethod
    def corner_triangles(cls, index: int, radius: float=.25):
        o = Box(radius*np.sqrt(2)).rotate(45)
        if index & cls.TL:
            yield o.translate((0, 0))
        if index & cls.TR:
            yield o.translate((0, 1))
        if index & cls.BL:
            yield o.translate((1, 0))
        if index & cls.BR:
            yield o.translate((1, 1))


def render_set(
    renderer, 
    tiles=Tiles.EDGE_AND_CORNER_ORDER,
    shape=(128, 128), 
    padding=0,
):
    images = []
    for row in tiles:
        for i in row:
            image = renderer(i, shape=shape)
            images.append(VF.to_pil_image(image))
        
    return VF.to_pil_image(make_grid([VF.to_tensor(i) for i in images], nrow=len(tiles[0]), padding=padding))

def renderer(index: int, shape=(128, 128)):
    obj = SmoothUnion(
        *Tiles.edges(index, radius=.2),
        *Tiles.corners(index, radius=.2),
        *Tiles.edge_circles(index, radius=.2),
        radius=.1,
    )#.sin_warp(freq=11, amount=.1/17)
    obj_n1 = obj.noise_warp(amount=1/100, freq=7)
    
    #mask = obj.round(.0).render_mask(shape=shape, abs=True, radius=.05)
    #norm1 = obj.render_mask_normal(shape=shape, abs=False, radius=.5)
    #norm2 = obj.round(.5).render_mask_normal(shape=shape, abs=True, radius=.6, z=10, mask_func=lambda m: gain(m, 5))
    #norm = mix(norm1, norm2[..., :2], 1)
    
    mask = step(calc_light(
        obj_n1.render_mask_normal(shape=shape, abs=False, radius=.2, z=10)
        , (-1, 0)), 0, -1)

    i1 = np.ones((*mask.shape, 3)) * np.array([[[.3, .8, .3]]])
    i2 = np.ones((*mask.shape, 3)) * np.array([[[.5, .4, .0]]])
    image = mix(i1, i2, mask)

    norm1 = obj_n1.render_mask_normal(
        shape=shape, abs=False, radius=.2, z=10,
        mask_func=lambda m: gain(m, 3)
    )
    light1 = calc_light(norm1, (0, -1))
    light2 = calc_light(
        obj_n1.render_mask_normal(
            shape=shape, abs=False, radius=.05, z=10,
            mask_func=lambda m: gain(m, 3)
        )
        , (-1, -.3)
    )
    light = mix(light1, light2, step(norm1[..., 0], 0, -1))[..., None]
    #mask = obj.render_mask(shape=shape)
    image += .5 * light.clip(0, 1)
    #image = mix(image, light * .5 + .5, .5)
    return image.clip(0, 1)
    
render_set(renderer, padding=1, shape=(64, 64))

In [ ]:
def edge_triangles(index: int, radius: float=.25, radius2: float = .5):
    cls = Tiles
    yield from cls.edge_triangles(index, radius=radius)
    o = Box(radius*2*np.sqrt(2)).rotate(45)
    if index & cls.ALL_EDGES == cls.ALL_EDGES:
        yield Box(1).translate(.5)
    if index & (cls.L|cls.T) == (cls.L|cls.T):
        yield o.translate((0, 0))
    if index & (cls.L|cls.B) == (cls.L|cls.B):
        yield o.translate((1, 0))
    if index & (cls.R|cls.T) == (cls.R|cls.T):
        yield o.translate((0, 1))
    if index & (cls.R|cls.B) == (cls.R|cls.B):
        yield o.translate((1, 1))

tiles_template = Tiles.EDGE_AND_CORNER_ORDER
def iter_objects(index: int): 
    yield from Tiles.edges(index, radius=.2)
    #yield from edge_triangles(index) 
    yield from Tiles.corners(index, radius=.2)
    yield from Tiles.edge_circles(index, radius=.2)

def renderer_mask(index: int, shape=(128, 128)):
    obj = Union(*iter_objects(index))
    image = obj.round(-.3).render_mask(radius=.4, shape=shape)
    return image.clip(0, 1)

def renderer_normal(index: int, shape=(128, 128)):
    obj = Union(*iter_objects(index))
    light = calc_light(
        obj.round(0).render_mask_normal(radius=-.1, shape=shape),
        (1, 1),
    )
    image = .5 + .2 * light
    return image.clip(0, 1)

render_set(renderer_normal, tiles=tiles_template, padding=1, shape=(32, 32))
#render_set(renderer_mask, tiles=tiles_template, padding=1, shape=(32, 32))
render_set(renderer_mask, tiles=tiles_template, padding=0, shape=(16, 16))

In [ ]:
def edge_triangles(index: int, radius: float=.25, radius2: float = .5):
    cls = Tiles
    yield from cls.edge_triangles(index, radius=radius)
    o = Box(radius*2*np.sqrt(2)).rotate(45)
    if index & cls.ALL_EDGES == cls.ALL_EDGES:
        yield Box(1).translate(.5)
    if index & (cls.L|cls.T) == (cls.L|cls.T):
        yield o.translate((0, 0))
    if index & (cls.L|cls.B) == (cls.L|cls.B):
        yield o.translate((1, 0))
    if index & (cls.R|cls.T) == (cls.R|cls.T):
        yield o.translate((0, 1))
    if index & (cls.R|cls.B) == (cls.R|cls.B):
        yield o.translate((1, 1))

tiles_template = np.tile(np.array(Tiles.EDGE_ORDER), (2, 2))
def iter_objects(index: int): 
    #yield from Tiles.edges(index, radius=.2)
    yield from edge_triangles(index) 
    #yield from Tiles.corners(index, radius=.2)
    #yield from Tiles.edge_circles(index, radius=.2)

def renderer_mask(index: int, shape=(128, 128)):
    obj = Union(*iter_objects(index))
    image = obj.round(-.3).render_mask(radius=.4, shape=shape)
    return image.clip(0, 1)

def renderer_normal(index: int, shape=(128, 128)):
    obj = Union(*iter_objects(index))
    light = calc_light(
        obj.round(0).render_mask_normal(radius=-.1, shape=shape),
    )
    image = .5 + .2 * light
    return image.clip(0, 1)

render_set(renderer_normal, tiles=tiles_template, padding=1, shape=(64, 64))
#render_set(renderer_mask, tiles=tiles_template, padding=1, shape=(64, 64))

In [ ]:
resize(render_set(renderer_normal, tiles=tiles_template, padding=0, shape=(16*4, 16*4)), 1/4)
#render_set(renderer_mask, tiles=tiles_template, padding=0, shape=(16, 16))